# 132: Stroke Progression + PQTL

Reminder, you can run something like:

`jupyter nbconvert --to html yourNotebook.ipynb`


# Manhattan Plots:

## First AIS (Existing Published GWAS vs. our meta-analysed GWAS)

![image](scratch/results/existing_stroke_gwas.png)
![image](scratch/results/meta_analysis/first_ais_metal_metaanalysis_manhattan.png)

Comparing by eye, they align quite well, with the existing stroke GWAS having much better results due to high power. HOWEVER, taking the top hitting 30 SNP results don't match  well with our results.  Therefore, I downloaded the full summary stats, and am looking at the wider range of hits.

## Subsequent GWASes (meta-analysed only)

![image](scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis_manhattan.png)
![image](scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis_manhattan.png)

## Subsequent AIS GWASes (adjusted for collider bias, then meta-analysed)
![image](scratch/results/subsequent_ais_metaanalysis_slopehunter1.txt.png)
![image](scratch/results/subsequent_ais_metaanalysis_dudbridge1.txt.png)
## Subsequent MACE GWASes (adjusted for collider bias, then meta-analysed)
![image](scratch/results/subsequent_mace_metaanalysis_slopehunter1.txt.png)
![image](scratch/results/subsequent_mace_metaanalysis_dudbridge1.txt.png)

In [ ]:
source("gwas_data_helpers.R")
library(dplyr)
gwas_columns <- c("MARKER", "BETA", "SE", "P")

first_ais_metaanalysis <- data.table::fread("scratch/results/meta_analysis/first_ais_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_ais_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_mace_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis.tbl", select = gwas_columns)

subsequent_ais_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_slopehunter1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_slopehunter1.txt", select = gwas_columns)
subsequent_ais_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_dudbridge1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_dudbridge1.txt", select = gwas_columns)

existing_stroke_gwas <- data.table::fread("scratch/data/existing_stroke_gwas.tsv.gz")
existing_stroke_gwas <- existing_stroke_gwas %>% dplyr::rename("CHR" = "chromosome", "BP" = "base_pair_location", "P" = "p_value",
                                                               "BETA" = "beta", "OR" = "odds_ratio", "LB" = "ci_lower", "UB" = "ci_upper")

existing_stroke_gwas$MARKER <- paste(existing_stroke_gwas$CHR, existing_stroke_gwas$BP, sep=":")
existing_stroke_gwas <- convert_odds_ratio_to_beta_estimate(existing_stroke_gwas)
existing_stroke_gwas_top_hits <- existing_stroke_gwas[existing_stroke_gwas$P < 1e-8,]

merge_gwases <- function(gwas) {
    merged_stroke_data_comparison <- merge(gwas, subsequent_ais_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("EXISTING_BETA" = "BETA.x", "EXISTING_P" = "P.x", "EXISTING_SE" = "SE.x",
                                                                                     "SUB_AIS_BETA" = "BETA.y", "SUB_AIS_P" = "P.y", "SUB_AIS_SE" = "SE.y")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_SH_BETA" = "BETA", "SUB_AIS_SH_P" = "P", "SUB_AIS_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_DB_BETA" = "BETA", "SUB_AIS_DB_P" = "P", "SUB_AIS_DB_SE" = "SE")    

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_BETA" = "BETA", "SUB_MACE_P" = "P", "SUB_MACE_SE" = "SE")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_SH_BETA" = "BETA", "SUB_MACE_SH_P" = "P", "SUB_MACE_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_DB_BETA" = "BETA", "SUB_MACE_DB_P" = "P", "SUB_MACE_DB_SE" = "SE")

    return(merged_stroke_data_comparison)
}

## Comparing p-values across all the different GWASes

There are two interesting things to notice from this table:
* In general, the subsequent MACE hits seem to align more closely with the incidence than the subsequent AIS does.
* SlopeHunter correction decimates the p-values of the MACE hits that show promise.



In [10]:
merged_existing_gwas <- merge_gwases(existing_stroke_gwas_top_hits)
merged_existing_gwas <- merge(merged_existing_gwas, first_ais_metaanalysis, by = "MARKER")
merged_existing_gwas <- merged_existing_gwas %>% dplyr::rename("FIRST_BETA" = "BETA", "FIRST_P" = "P", "FIRST_SE" = "SE")

merged_existing_gwas <- transform(merged_existing_gwas, SUB_AIS_MIN_P_VALUE = pmin(SUB_AIS_SH_P, SUB_AIS_DB_P))
merged_existing_gwas <- transform(merged_existing_gwas, SUB_MACE_MIN_P_VALUE = pmin(SUB_MACE_SH_P, SUB_MACE_DB_P))

print("Subsequent AIS against the Incidence Top Hits (non corrected)")
pruned_ais_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_AIS_P")
head(pruned_ais_snps[with(pruned_ais_snps, order(SUB_AIS_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")])

print("Subsequent AIS against the Incidence Top Hits (corrected)")
pruned_ais_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_AIS_MIN_P_VALUE")
head(pruned_ais_snps[with(pruned_ais_snps, order(SUB_AIS_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")])

print("Subsequent MACE against the Incidence Top Hits (non corrected)")
pruned_ais_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_MACE_P")
head(pruned_ais_snps[with(pruned_ais_snps, order(SUB_MACE_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")])

print("Subsequent MACE against the Incidence Top Hits (corrected)")
pruned_mace_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_MACE_MIN_P_VALUE")
head(pruned_mace_snps[with(pruned_mace_snps, order(SUB_MACE_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")])

[1] "Subsequent AIS against the Incidence Top Hits (non corrected)"


MARKER,EXISTING_P,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7:92243719,3.114e-09,3.784e-01,0.0005098,0.0004415,0.0001885
9:22124450,1.167e-11,1.181e-05,0.0004771,0.0053670,0.0004495
4:155524009,1.972e-10,8.933e-02,0.0002233,0.0039340,0.0012800
2:203650410,8.798e-10,1.216e-01,0.0036200,0.0044720,0.0016570
6:49425521,1.656e-09,2.385e-02,0.0011480,0.0098700,0.0022590
1:3221083,5.952e-10,3.360e-02,0.0033070,0.0062260,0.0026920


[1] "Subsequent AIS against the Incidence Top Hits (corrected)"


MARKER,EXISTING_P,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7:92243719,3.114e-09,3.784e-01,0.0005098,0.0004415,0.0001885
9:22124450,1.167e-11,1.181e-05,0.0004771,0.0053670,0.0004495
4:155524009,1.972e-10,8.933e-02,0.0002233,0.0039340,0.0012800
2:203650410,8.798e-10,1.216e-01,0.0036200,0.0044720,0.0016570
6:49425521,1.656e-09,2.385e-02,0.0011480,0.0098700,0.0022590
1:3221083,5.952e-10,3.360e-02,0.0033070,0.0062260,0.0026920


[1] "Subsequent MACE against the Incidence Top Hits (non corrected)"


MARKER,EXISTING_P,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9:22114495,1.400e-09,0.000117,0.0018450,0.007047,0.0008152
6:49385182,1.904e-09,0.022680,0.0018950,0.011560,0.0026440
12:112849899,3.996e-09,0.033770,0.0732800,0.504900,0.3157000
4:155524009,1.972e-10,0.089330,0.0002233,0.003934,0.0012800
16:87561696,2.290e-09,0.039690,0.2954000,0.010800,0.0293700
12:112463296,7.492e-10,0.018960,0.1484000,0.771300,0.5111000


[1] "Subsequent MACE against the Incidence Top Hits (corrected)"


MARKER,EXISTING_P,FIRST_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9:22115026,6.147e-12,0.0001093,0.0004201,0.0002852,1.377e-05
4:155502869,4.625e-17,0.0496200,0.0226100,0.0111200,4.359e-03
7:92253972,8.081e-09,0.9222000,0.0246100,0.0133500,1.332e-02
6:49345141,2.566e-09,0.0135400,0.0209500,0.0242400,1.617e-02
13:110834217,4.339e-09,0.0115500,0.0241400,0.0264200,1.994e-02
16:87562316,2.317e-09,0.0530700,0.3454000,0.0207300,2.474e-02


Look at the above in comparison with the starting point of the meta analysed incidence GWAS that we have available from our data...

Interesting thing to note here:
* The Dudbridge Correction seemed to increase most of these hits...

In [8]:
significant_first_ais_comparison <- first_ais_metaanalysis[first_ais_metaanalysis$P < 1e-8,]
significant_first_ais_comparison <- merge_gwases(significant_first_ais_comparison)

significant_first_ais_comparison <- transform(significant_first_ais_comparison, SUB_AIS_MIN_P_VALUE = pmin(SUB_AIS_P, SUB_AIS_SH_P, SUB_AIS_DB_P))
significant_first_ais_comparison <- transform(significant_first_ais_comparison, SUB_MACE_MIN_P_VALUE = pmin(SUB_MACE_P, SUB_MACE_SH_P, SUB_MACE_DB_P))

print("Subsequent AIS Top Hits (when filtered by existing incidence GWAS 1e-8)")
pruned_ais_snps <- basic_snp_pruning(significant_first_ais_comparison, p_value_column="SUB_AIS_MIN_P_VALUE")
head(pruned_ais_snps[with(pruned_ais_snps, order(SUB_AIS_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")])

print("Subsequent MACE Top Hits (when filtered by existing incidence GWAS 1e-8)")
pruned_mace_snps <- basic_snp_pruning(significant_first_ais_comparison, p_value_column="SUB_AIS_MIN_P_VALUE")
head(pruned_mace_snps[with(pruned_mace_snps, order(SUB_MACE_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")])

[1] "Subsequent AIS Top Hits (when filtered by existing incidence GWAS 1e-8)"


MARKER,EXISTING_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
9:22099746,2.156e-09,0.001817,0.20280,0.002616
2:203865822,5.581e-09,0.010870,0.07700,0.025080
7:92244630,7.712e-09,0.010930,0.16100,0.035380
7:92257602,2.963e-09,0.010990,0.08762,0.021230
2:203888355,6.357e-09,0.011190,0.14450,0.026710
7:92263789,2.594e-09,0.020800,0.06938,0.026960


[1] "Subsequent MACE Top Hits (when filtered by existing incidence GWAS 1e-8)"


MARKER,EXISTING_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
9:22099746,2.156e-09,1.748e-06,7.168e-01,0.0001131
1:183098697,6.978e-09,2.255e-01,6.335e-05,0.0967600
1:183005231,4.449e-09,4.520e-01,1.301e-04,0.0962000
1:183105855,2.742e-09,5.511e-01,5.065e-04,0.1396000
9:22085719,6.095e-09,5.295e-04,5.081e-01,0.0057460
1:183115328,8.152e-10,5.623e-01,5.938e-04,0.1743000


## Working Backwards from the Subsequent GWAS Results



In [35]:
significant_subsequent_ais_comparison <- subsequent_ais_metaanalysis_not_adjusted[subsequent_ais_metaanalysis_not_adjusted$P < 1e-5,]
significant_subsequent_ais_comparison <- merge_gwases(significant_subsequent_ais_comparison)
significant_subsequent_ais_comparison <- merge(significant_subsequent_ais_comparison, existing_stroke_gwas, by = "MARKER")
significant_subsequent_ais_comparison <- significant_subsequent_ais_comparison %>% dplyr::rename("ORIG_EXISTING_BETA" = "BETA", "ORIG_EXISTING_P" = "P", "ORIG_EXISTING_SE" = "SE")
significant_subsequent_ais_comparison <- basic_snp_pruning(significant_subsequent_ais_comparison, p_value_column="SUB_AIS_P")

print("Subsequent AIS Top Hits")
significant_subsequent_ais_comparison[, c("MARKER", "ORIG_EXISTING_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")]

significant_subsequent_mace_comparison <- subsequent_mace_metaanalysis_not_adjusted[subsequent_mace_metaanalysis_not_adjusted$P < 1e-5,]
significant_subsequent_mace_comparison <- merge_gwases(significant_subsequent_mace_comparison)
significant_subsequent_mace_comparison <- merge(significant_subsequent_mace_comparison, existing_stroke_gwas, by = "MARKER")
significant_subsequent_mace_comparison <- significant_subsequent_mace_comparison %>% dplyr::rename("ORIG_EXISTING_BETA" = "BETA", "ORIG_EXISTING_P" = "P", "ORIG_EXISTING_SE" = "SE")
significant_subsequent_mace_comparison <- basic_snp_pruning(significant_subsequent_mace_comparison, p_value_column="SUB_AIS_P")

print("Subsequent MACE Top Hits")
significant_subsequent_mace_comparison[, c("MARKER", "ORIG_EXISTING_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")]

MARKER,ORIG_EXISTING_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
10:14083037,0.91500,7.092e-06,2.319e-04,4.860e-04
10:81136015,0.03442,5.888e-06,2.592e-05,1.792e-05
12:108666175,0.35600,4.522e-06,2.042e-05,1.537e-05
13:52428066,0.01186,8.997e-06,1.517e-05,8.119e-06
13:62059558,0.92770,5.973e-06,3.571e-05,5.558e-05
14:62708880,0.13560,8.852e-06,5.191e-05,1.359e-04
14:73294871,0.30270,2.290e-06,1.666e-05,6.377e-06
16:4417797,0.27940,9.764e-07,4.465e-03,2.457e-03
1:158889221,0.06908,9.732e-06,5.954e-04,2.916e-04


MARKER,ORIG_EXISTING_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
13:112784083,9.792e-01,1.007e-06,4.892e-01,4.212e-01
13:62079916,8.618e-01,6.969e-06,3.004e-03,6.652e-03
16:4417797,2.794e-01,4.132e-07,4.926e-02,1.706e-02
19:34198233,6.495e-02,4.396e-06,6.797e-03,1.849e-02
20:8007069,6.586e-02,5.828e-06,4.147e-01,3.770e-01
3:191782060,6.456e-01,7.844e-06,4.300e-02,7.286e-02
4:90752646,4.713e-01,8.297e-06,1.810e-01,2.051e-01
5:40389287,5.477e-01,9.824e-06,1.079e-01,1.889e-01
6:136685475,4.099e-01,9.218e-06,7.469e-01,9.401e-01


# Forest Plots of Potentially Interesting SNPs
. | . 
- | - 
![image](scratch/results/9_22099746.png) | ![image](scratch/results/9_22124450.png)
![image](scratch/results/1_183098697.png) | ![image](scratch/results/4_155502869.png)